# <ins> Applied Data Science Capstone Course </ins>

# Capstone Project - The Battle of the Neighbrhoods (Week 5) - Part 2

## <ins> *Identifying Unique Local Venues in Toronto* </ins>

# 1. Introduction

### 1.1 Background

Toronto is the largest and the most populous city in Canada, with a population of nearly 3 million people. It is one of the most diverse and multicultural metropolitan areas in the world. It is also one of Canada's leading tourism destinations and has a rich cultural life. In 2017, the Toronto-area received 44 million tourists, of whom over 10 million were domestic visitors and 3 million were from the United States, spending a total of $8.84 billion. 

As is generally the case with such cities, Toronto has a rich food culture that includes a vast array of international cuisines influenced by the city's diverse demographics. It is home to tens of thousands of restaurants and several hundreds of thousands of independent as well as chain coffee shops, bakeries, food stores, etc.  
As such, when tourists make plans to visit Toronto or new residents from different parts of Canada or immigrants from other countries plan their moves to Toronto, there is quite a lot of research material available with regards to popular food and tourist destinations. However, some of the other venues that are an integral part of people’s day-to-day lives receive less focus and attention. Some examples of these are pet stores, local outdoor recreation facilities, fitness centers, to name a few. 

So, this project will attempt to identify top local venues in four preselected venue categories, i.e., fitness, pets, outdoor recreation, and transportation. It will exclude restaurants and other food-related venues returned as a part of the venue analysis, as this data is not relevant to the problem statement being addressed by this project. Additionally, it will limit the number of boroughs and/or neighborhoods that will be considered in scope, to run a practical and meaningful analysis.

The project will utilize data science tools to analyze Toronto neighborhood data and venue data from Foursquare API, and focus on four venue types listed above, and will aim to determine the presence and distribution of venues of these four types across the neighborhoods of Toronto.  


### 1.2 Business Problem

The aim of this project is __to find venues and local establishments in Toronto, Canada, that specifically fall into one of the four select categories, i.e., fitness, pets, outdoor recreation, and transportation.__ These venue types were selected keeping in mind the target stakeholders such as people who have newly moved to Toronto and are looking to find local spots that they can frequent as a part of their new routine, as these locations are not commonly covered in tourist brochures or materials. The project aims to solve this problem by leveraging data on Toronto neighborhood venues, their types, frequencies, etc., thus, providing beneficial information to people looking for a unique experience in a large and diverse city, such as Toronto.

### 1.3 Interest

While a lot of information is readily available on top restaurants and common tourist attractions, people who have newly moved to the city of Toronto from different parts of Canada or from international locations, for temporary assignments or permanent relocation, can greatly benefit from finding local spots like gyms/fitness centers, bookshops, pet stores etc. which are not commonly featured in visitor resources. It can also benefit tourists and visitors who may be interested in checking out local establishments.

## 2. Data Acquisition and Cleaning

### 2.1 Data Sources

This assignment requires exploring, segmenting, and clustering the neighborhoods in the city of Toronto. While this neighborhood data is not readily available on the internet, a <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"> Wikipedia</a> page exists for the Toronto neighborhood data, that has all the information needed to explore and cluster the neighborhoods in Toronto. This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Specifically, this table of Toronto postal codes also contains the names of Boroughs and Neighborhoods in Toronto. 

Geographical coordinates for each of the Toronto postal codes will be extracted from a csv file: <a href="http://cocl.us/Geospatial_data"> Toronto Geospatial Data</a>

Finally, venue categories and details required for the last part of the analysis will obtained via Foursquare API.

### 2.2 Data cleaning

Once the dataset is scraped from the above Wikipedia page, it will need to be wrangled, cleaned, and read into a pandas dataframe so that it is in a structured format for further analysis.

After reading the data from the Wikipedia page, we notice that the headers, Postal code, Borough, and Neighborhood have trailing new line characters (“\n”), which need to be removed. Also, each of the data fields have a similar issue, where they have a trailing new line character. Therefore, these fields are also cleaned to ensure a consistent format. 

Additionally, there are records where the Boroughs are “Not Assigned”. For the purpose of this analysis, these records need to be dropped. Similarly, there are records where the Neighborhood column is blank. To address this, we identify such records and assign the value of the Borough field to the corresponding Neighborhood field, for consistency.

Once the data is in a structured format, we can start the analysis of the dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
# Importing necessary libraries

import requests
import lxml.html as lh
import pandas as pd
import numpy as np

### Obtaining data from the provided Wikipedia page and transforming into a pandas dataframe

In [2]:
#Link to the datasource
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Creating a handle, page, to handle the contents of the website
page = requests.get(url)

#Storing the contents of the website under doc
doc = lh.fromstring(page.content)

#Parsing data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

### 2.3 Feature Selection

After the first step of data cleaning described above, there were 181 samples and three features in the dataset. After completing the second step of data cleaning, that is, dropping rows where the Borough field has a “Not Assigned” value, we end up with 104 samples (and three features).

Upon examining each of the three features, it is clear that all three features present in the dataset, *Postal code, Borough, and Neighborhood*, are essential to our analysis. Therefore, we proceed to the next step with these features.

### Checking if the dataframe consists of three columns: PostalCode, Borough, and Neighborhood

In [3]:
#Checking the length of the first few rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:
# Parsing the first row as the header
tr_elements = doc.xpath('//tr')

#Creating empty list
col=[]
i=0

#For each row, store each first element (header) in an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


## 3. Methodology

### 3.1 Explore Dataset

After reading the data from the Wikipedia page and cleaning it, it is transformed into a pandas dataframe. Before performing this step, we import necessary libraries such as pandas and numpy. 
This dataframe captures the three essential features discussed above: Postal code, Borough, and Neighborhood.

To prepare the dataset for further analyses, as previously described in the section above, (1) we drop records where the Borough field has a “Not Assigned” value, and, (2) assign the value from the Borough column to the Neighborhood column, where the Neighborhood field is blank.

Finally, the Neighborhood records are grouped based on Postal codes and Boroughs, resulting in a clean and consolidated dataset which is conducive to performing the next step, i.e., exploring the neighborhoods in Toronto.

Then, we need to use geopy package to get the geographical coordinates of the Toronto neighborhoods. However, this package can sometimes function in an unreliable fashion and may require multiple calls to get the geographical coordinates of the neighborhoods. 
Therefore, this project used a .csv file that has the geographical coordinates of each postal code found in Toronto: http://cocl.us/Geospatial_data
This data was read into a pandas dataframe, cleaned to ensure that it is ready for merging with the initial dataframe. After the data cleaning step, both dataframes were merged, such that the resulting dataframe contains geographical coordinates (latitude, longitude) for each postal code in the original dataframe. This merged dataframe now has five columns or features: Postal code, Borough, Neighborhood, Latitude, and Longitude.

In the next step, we import and use the folium visualization library in conjunction with the geographical coordinates from the above step, to create a map of Toronto neighborhoods. Note: to reduce complexity, this project takes into account only those boroughs that contain the term “Toronto”.

### Creating the pandas dataframe

In [ ]:
#Each header needs to be appended to a tuple along with an empty list

#Since the first row is the header row, data is stored from the second row onwards
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    #If length of the row is not 3, then //tr data is not from the table 
    if len(T)!=3:
        break
   
    #i is the index of the column
    i=0
    
    #Iterating through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Checking if row is empty
        if i>0:
        #Converting any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Appending the data to the empty list of the i'th column
        col[i][1].append(data)
       
        i+=1

In [ ]:
# Checking the length of each column
[len(C) for (title,C) in col]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


### Cleaning the data in the dataframe

In [9]:
df = df.replace('\n',' ', regex=True)

df.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### Dropping "Not assigned" Borough rows

In [10]:
df=df[~df["Borough\n"].str.contains("Not assigned")]

# Resetting the index and dropping the previous index
df=df.reset_index(drop=True)

df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Grouping by neighborhoods based on Postal codes and Boroughs

In [11]:
#Grouping neighborhoods by postal codes and consolidating into single records
df = df.groupby(['Postal code\n', 'Borough\n'])['Neighborhood\n'].apply(','.join).reset_index()
#df.columns = ['Postal code\n','Borough\n','Neighborhood\n']

df['Neighborhood\n'] = df['Neighborhood\n'].str.replace('/',',')

df.head(100)

,Postal code,Borough,Neighborhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern , Rouge"
2,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
3,M1E,Scarborough,"Guildwood , Morningside , West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
8,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
9,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village W..."


### Replacing 'Not assigned' neighborhoods with the name of the Borough

In [12]:
df['Neighborhood\n'] = df['Neighborhood\n'].str.strip()
df.loc[df['Neighborhood\n'] == 'Not assigned', 'Neighborhood\n'] = df['Borough\n']
df.head()

,Postal code,Borough,Neighborhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern , Rouge"
2,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
3,M1E,Scarborough,"Guildwood , Morningside , West Hill"
4,M1G,Scarborough,Woburn


In [13]:
#Dropping unnecessary rows
df=df[~df["Borough\n"].str.contains("Canadian postal codes")]
df=df.reset_index(drop=True)
df.head(10)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


### Printing the number of rows of the dataframe using Shape method

In [14]:
#Checking the shape of the dataframe
df.shape

(103, 3)

In [15]:
#Importing geopy to attempt getting geographical data
import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

### Reading Geospatial data from the csv file as the functioning of geocoder can be unreliable

In [16]:
# Reading the cvs file and convert it to a dataframe

url='http://cocl.us/Geospatial_data'
df_postalcodes=pd.read_csv(url)
df_postalcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Renaming Postal Code column

In [17]:
df_postalcodes.columns = ['Postal code', 'Latitude', 'Longitude']
df_postalcodes.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df.columns = ['Postal code', 'Borough', 'Neighborhood']
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Sorting dataframe by postal codes and merging with values read from the csv file

In [19]:
dfs=df.sort_values(by='Postal code', ascending=True)
dfs.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Cleaning main dataframe values to ensure a clean merge with the other dataframe (postalcodes, latitude, and longitude)

In [20]:
dfs['Postal code'] = dfs['Postal code'].str.strip()
dfs['Borough'] = dfs['Borough'].str.strip()
dfs['Neighborhood'] = dfs['Neighborhood'].str.strip()


### Merging columns into the dataframe

In [21]:
geo_df=pd.merge(dfs,df_postalcodes, how='right', on='Postal code')
geo_df.head(15)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


### Checking the dimensions of the new dataframe

In [22]:
geo_df.shape

(103, 5)

## Exploring and clustering the neighborhoods in Toronto

### Importing necessary libraries

In [23]:
#Importing necesssary libraries
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

In [24]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt



In [25]:
import requests # for handling requests
from pandas.io.json import json_normalize # for tranforming JSON files into pandas dataframes

In [26]:
#Importing libraries for plotting neighborhood data
import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
#Importing k-means for clustering stage
from sklearn.cluster import KMeans

In [28]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # for map rendering

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

## Please note that I have chosen the option of working with only boroughs that contain the word Toronto

In [29]:
toronto_data= geo_df[geo_df['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [30]:
toronto_data.shape

(39, 5)

### There are 39 boroughs with the word 'Toronto' 

### Getting the coordinates (latitude, longitude) of Toronto using Google

In [31]:
latitude = 43.6532
longitude= -79.3832

### Creating a map of Toronto using folium

In [32]:
#Creating a map of Toronto using latitude and longitude values above
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

#Adding markers to the map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2	Explore Neighborhoods in Toronto

In this stage, the Foursquare API will be utilized to explore the neighborhoods in Toronto and segment them. For this, it is required to define our Foursquare credentials and version as a starting point. We then get a neighborhood’s name and geographical coordinates to start the neighborhood exploration process. We then set a limit on the number of venues returned by the Foursquare API to 100 as well as define a radius of 500 meters. Once the API returns the results in the form of a JSON file, it is cleaned and structured into a pandas dataframe.

We can create a user defined function, getNearbyVenues, to repeat the above process for all neighborhoods in Toronto. 226 unique venue categories were identified across 39 Toronto boroughs as a result of this process.

## Utilizing the Foursquare API to explore Toronto neighborhoods

### Foursquare credentials

In [33]:
CLIENT_ID = 'HXKV4HYMACHQQBRQEGZG2STQTISDV40D2D3LICOKY2RSNRDT' # Foursquare ID
CLIENT_SECRET = 'UNMYSSD1W5RDUDELPXREOFL40AWYKAT2CSGP41DP1CQ3ODDC' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # Defining a raius

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       #Creating the API request URL 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #Making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Returning only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Running the above code for different Toronto neighborhoods

In [36]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(1623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


### Checking venues across neighborhoods

In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton , Parkdale Village , Exhibition Place",24,24,24,24,24,24
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


### Categorizing the venues returned above

In [39]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 unique categories.


### 3.3 Analyze Each Neighborhood

Using one-hot encoding technique, we further analyze each neighborhood in the boroughs of Toronto city. Next, rows are grouped by neighborhood, by taking the mean of the frequency of occurrence of each venue category. 

In the next step, the top 5 venue categories can be printed out for each neighborhood, which is a crucial part of this project. These should then be converted into a pandas dataframe for the next step of the analysis.

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# moving neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [41]:
toronto_onehot.shape

(1623, 226)

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.017241,0.034483,0.000000,0.000000,0.00,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.017241,0.051724,0.068966,0.000000,0.000,0.000000,0.00,0.000000,0.017241,0.000000,0.017241,0.000000,0.00,0.017241,0.000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.00,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.01724

In [43]:
toronto_grouped.shape

(39, 226)

### Printing top venues for each neighborhood

In [44]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2        Bakery  0.03
3      Beer Bar  0.03
4    Restaurant  0.03


----Brockton , Parkdale Village , Exhibition Place----
                   venue  freq
0                   Café  0.12
1            Coffee Shop  0.08
2         Breakfast Spot  0.08
3              Nightclub  0.08
4  Performing Arts Venue  0.04


----Business reply mail Processing CentrE----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2              Garden  0.06
3                Park  0.06
4          Comic Shop  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.

### Transforming into a dataframe

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Café
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Performing Arts Venue,Bar,Music Venue,Burrito Place,Grocery Store,Gym
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Burrito Place,Restaurant,Brewery,Smoke Shop
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Salad Place,Fried Chicken Joint,Restaurant
5,Christie,Grocery Store,Café,Park,Baby Store,Diner,Nightclub,Candy Store,Gas Station,Coffee Shop,Restaurant
6,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant,Pub,Gastropub,Hotel,Yoga Studio,Men's Store
7,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
8,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Coffee Shop,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Bar
9,Davisville North,Dance Studio,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Department Store,Food & Drink Shop,Donut Shop,Doner Restaurant


### 3.4 Cluster neighborhoods

Once the dataframe with Toronto neighborhoods and their top venue categories is created, we use the k-means Clustering algorithm, for clustering Toronto neighborhoods. The k-value was set at 5, and the algorithm was run to cluster the Toronto neighborhoods into 5 clusters.

We then create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood, as shown below.

Finally, these neighborhood clusters are visualized with the help of folium visualization library as described in a previous section.

In [47]:
# setting the number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [48]:
toronto_merged = toronto_data

# adding clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merging toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Park,Pub,Trail,Health Food Store,Yoga Studio,Diner,Deli / Bodega,Department Store,Dessert Shop,Distribution Center
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Cosmetics Shop,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Pizza Place,Sandwich Place,Italian Restaurant,Restaurant,Steakhouse,Ice Cream Shop,Board Shop,Brewery,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Pet Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


### Visualization of Toronto neighborhood clusters

In [49]:
# creating Toronto neighborhood map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.5 Examine Clusters

In this stage, each of the five neighborhood clusters are examined to determine the venue categories that distinguish each cluster from the others. Based on the defining categories, different names have been assigned to each of these neighborhood clusters in Toronto.

For example, in the first neighborhood cluster, the key venue categories are restaurants and coffee shops while the third cluster has a unique venue category of airport that distinguishes it from all other clusters.


## Examining different neighborhood clusters in Toronto

### Cluster 1: Restaurants and Coffee Shops

When k-means clustering algorithm was applied, with k=5, the first cluster returned was the following. The Toronto boroughs covered in this cluster include, East Toronto, Central Toronto, Downtown Toronto, and West Toronto. As is evident from the dataframe included below, while setting aside the most popular venue categories of ‘restaurants and coffee shops’, this neighborhood offers popular alternate venues of the four preselected venue types as follows:

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Park,Pub,Trail,Health Food Store,Yoga Studio,Diner,Deli / Bodega,Department Store,Dessert Shop,Distribution Center
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Frozen Yogurt Shop,Cosmetics Shop,Brewery,Bubble Tea Shop
2,East Toronto,0,Fast Food Restaurant,Pizza Place,Sandwich Place,Italian Restaurant,Restaurant,Steakhouse,Ice Cream Shop,Board Shop,Brewery,Burrito Place
3,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Pet Store
4,Central Toronto,0,Park,Bus Line,Swim School,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
5,Central Toronto,0,Dance Studio,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Department Store,Food & Drink Shop,Donut Shop,Doner Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Rental Car Location,Spa,Restaurant,Chinese Restaurant,Toy / Game Store
7,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Gym,Coffee Shop,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Bar
8,Central Toronto,0,Playground,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
9,Central Toronto,0,Coffee Shop,Pub,Supermarket,Bank,Sports Bar,Sushi Restaurant,Pizza Place,Liquor Store,Fried Chicken Joint,Light Rail Station


### Cluster 2: Restaurants

The second cluster returned by the algorithm includes only one Toronto borough, i.e., Downtown Toronto. And the analysis of this cluster shows that all top venue categories provided by the Foursquare API for this neighborhood align with the restaurants or food places category. Thus, this cluster does not yield any results that are necessary to address the problem this project aims to solve. i.e., it does not return any venue that fall under one of the four chosen venue types.

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Salad Place,Deli / Bodega,American Restaurant,Asian Restaurant


### Cluster 3: Airport

The third cluster returned by the algorithm also includes only one Toronto borough, i.e., Downtown Toronto. The analysis of this cluster shows that this neighborhood includes the Airport, which falls in one of the four venue categories in scope for this project’s problem statement. The presence of Airport and relates service options in this neighborhood cluster makes it unique and differentiates it from all the other neighborhoods.

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,2,Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry


### Cluster 4: Restaurants and Pubs

Similar to the previous two clusters, the fourth cluster below also includes only one Toronto borough, i.e., Downtown Toronto. The analysis of this cluster shows that majority of the top venue categories returned by the Foursquare API for this neighborhood fall in venue categories that are not in scope for this project, i.e., restaurants/coffee shops/bars. However, there is one venue category, Yoga Studio, in this cluster that can help address the problem statement at hand.

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant,Pub,Gastropub,Hotel,Yoga Studio,Men's Store


### Cluster 5: Restaurants and Cafe's

Since the k in the k-means algorithm used for this project was set at 5, the following is the final cluster that was analyzed for this project. It covers two neighborhoods within Downtown Toronto borough. Analyzing this final cluster provides one meaningful result which is tabulated below.

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Downtown Toronto,4,Coffee Shop,Café,Restaurant,American Restaurant,Clothing Store,Hotel,Thai Restaurant,Gym,Deli / Bodega,Cosmetics Shop
26,Downtown Toronto,4,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dessert Shop,Gaming Cafe,Grocery Store,Bar,Vegetarian / Vegan Restaurant


## 4. Results

The objective of this project was to help stakeholders identify some of the unique local spots in the city of Toronto, that fall in one of the four select categories. This has been achieved by first scraping the Wikipedia page that contains the necessary neighborhood details for Toronto, Foursquare API to get different venue categories and venue details, and applying k-means clustering algorithm to segment and cluster the various neighborhoods in the city of Toronto.
By following the methodology described in the section above and performing an analysis of the Toronto neighborhood dataset, the following results were achieved.
As noted previously, only those boroughs containing ‘Toronto’ in their name were considered in scope for the purpose of this analysis. This resulted in 39 boroughs in Toronto being grouped into 5 clusters, for which 226 unique venue categories were returned by the Foursquare API. This section will discuss each of these 5 neighborhood clusters in detail and leverage the venue information returned by the analysis to address the questions this project set out to answer.

### Summary

The results obtained by analyzing the five neighborhood clusters returned by the k-means clustering algorithm are summarized here below. This project looks at four select categories: Fitness, Pets, Outdoor Recreation, and Transport. It does not include venue categories such as restaurants, coffee shops, and other food related venues. 

• Gyms and fitness centers (including yoga studios) emerged as the top venue type with even distribution across the four boroughs

• Transport facilities are concentrated primarily in Central Toronto and Downtown Toronto

• West Toronto ranks low with regards to the presence of outdoor recreation and transport facilities

## 5. Discussion

This project set out to identify unique local establishments that fall in four select venue categories: Fitness, Pets, Outdoor Recreation, and Transport. Segmenting and exploring different boroughs and neighborhoods in the city of Toronto by bringing together location data and venue data (Foursquare), and analyzing it with the help of a clustering algorithm such as k-means, helped find answers to the problem this project set out to solve. 

To discuss the results summarized above, while most top venues in a majority of the Toronto neighborhoods are restaurants, coffee shops, bars, and other food/drink places, we were able to identify other local establishments and locations that are offered by the different neighborhoods of the city, including gyms / fitness centers, yoga studios, dance studios, parks, dog runs, and pet stores that are critical to providing a fulfilling experience to Toronto residents (new and current) as well as visitors and tourists.

As information on restaurants and other food-related venues is more readily available than information on other types of venues, the above analysis uncovers the fact that there are several neighborhoods in a large and diverse metropolitan city like Toronto where other venue categories, such as the four selected for this project, consistently fall in the top venue categories. This information can be easily accessed by the temporary visitors and new residents of the city to make these locations a part of their new life or routine in the city of Toronto.


## 6. Conclusion

In conclusion, this project report explored and analyzed the different neighborhoods in the city of Toronto, Canada, using the postal codes, boroughs, neighborhoods, and geographical coordinates data with the objective of identifying popular local establishments of four preselected venue types. It segmented and clustered each of the Toronto neighborhoods with the help of k-means Clustering algorithm. And then studied the different venue categories, specifically focusing on popular yet unique local venues such as fitness centers, pet stores, etc., while excluding some of the conventionally popular venue types such as restaurants and coffee shops. It helped uncover insights on local spots that are not generally highlighted in other readily-available tourist resources and materials, and can be leveraged by new residents of this city as well as visitors to have a unique experience during their time in the city of Toronto.